In [1]:
import pandas as pd
import numpy as np
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv,find_dotenv
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

from operator import itemgetter
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from typing import List,Union, Dict
from langchain_core.messages import AIMessage
from copy import deepcopy

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [4]:
@tool
def multiply(firstInt:int, secondInt: int) -> int:
    """
        Multiply two integers togather
    """
    return firstInt*secondInt

@tool
def add(firstInt:int, secondInt: int) -> int:
    """
        Add two integers togather
    """
    return firstInt+secondInt

@tool 
def exponentiate (base:int, exponent:int) -> int:
    """
        Exponentiate the base to the exponent power
    """
    return base**exponent   

In [5]:
tools=[multiply,add,exponentiate]

In [6]:
llmWithTools=llm.bind_tools(tools=tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [7]:
multiply.name

'multiply'

In [8]:
multiply

StructuredTool(name='multiply', description='Multiply two integers togather', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002DA70B6BF60>)

In [9]:
toolMap={tool.name:tool for tool in tools}

In [10]:
toolMap

{'multiply': StructuredTool(name='multiply', description='Multiply two integers togather', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002DA70B6BF60>),
 'add': StructuredTool(name='add', description='Add two integers togather', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002DA0EC49E40>),
 'exponentiate': StructuredTool(name='exponentiate', description='Exponentiate the base to the exponent power', args_schema=<class 'langchain_core.utils.pydantic.exponentiate'>, func=<function exponentiate at 0x000002DA0EC49DA0>)}

In [11]:
llmWithTools.invoke(input="What's 23 to the power 7?").tool_calls  # toolCalls

{'name': 'multiply', 'description': 'Multiply two integers togather', 'parameters': {'type_': 6, 'description': 'Multiply two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['firstInt', 'secondInt'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'add', 'description': 'Add two integers togather', 'parameters': {'type_': 6, 'description': 'Add two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum':

[{'name': 'exponentiate',
  'args': {'exponent': 7.0, 'base': 23.0},
  'id': 'f22f3e02-e077-4979-9f9b-9e339c7bffd5',
  'type': 'tool_call'}]

In [12]:
def toolIdentifier(msg: AIMessage) -> Runnable:
    toolMap={tool.name:tool for tool in tools}
    for toolInfo in msg.tool_calls:
        toolInfo['output']=toolMap[toolInfo['name']].invoke(input=toolInfo['args'])
    return msg.tool_calls
        

In [13]:
chain=(llmWithTools | toolIdentifier) 

In [14]:
chain.invoke(input="What's 23 to the power 7?")

{'name': 'multiply', 'description': 'Multiply two integers togather', 'parameters': {'type_': 6, 'description': 'Multiply two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['firstInt', 'secondInt'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'add', 'description': 'Add two integers togather', 'parameters': {'type_': 6, 'description': 'Add two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum':

[{'name': 'exponentiate',
  'args': {'exponent': 7.0, 'base': 23.0},
  'id': 'fbaa1923-2535-4a72-a793-4fa8a1f2853f',
  'type': 'tool_call',
  'output': 3404825447}]

In [15]:
chain.invoke(input="What's 23 times 7?")

{'name': 'multiply', 'description': 'Multiply two integers togather', 'parameters': {'type_': 6, 'description': 'Multiply two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['firstInt', 'secondInt'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'add', 'description': 'Add two integers togather', 'parameters': {'type_': 6, 'description': 'Add two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum':

[{'name': 'multiply',
  'args': {'secondInt': 7.0, 'firstInt': 23.0},
  'id': '162faf9f-adf5-4bae-9d4a-c7412d6f6cc8',
  'type': 'tool_call',
  'output': 161}]